In [2]:
import csv
import pymongo
from pymongo import MongoClient
import datetime as dt
import calendar
import pandas as pd
import numpy as np
import time
from dateutil.relativedelta import relativedelta

In [3]:
Client = MongoClient("mongodb://3.20.189.28:2036/momoKashBd?readPreference=secondary&directConnection=true")
db_loan = Client["momoKashBd"]
#These are my collections
#Clients = db_v1["Client"]
#Loans = db_v1["Loan"]
Loans = db_loan["Loan"]
print("MongoDB connection established with success")

MongoDB connection established with success


In [4]:
# Start and end dates for iteration
start_date = dt.datetime(2023, 8, 1)  # Earliest (Aug 2023)
end_date = dt.datetime(2025, 4, 1)    # Latest (Apr 2025)

In [6]:
while current_date <= final_date:
    date_start = current_date
    date_end = date_start + relativedelta(months=1) - dt.timedelta(days=1)

    print(f"\nProcessing loans from {date_start.strftime('%B %Y')}...")

    start = time.time()

NameError: name 'current_date' is not defined

In [ ]:
# Quick test: check if any loanDate exists
cursor = Loans.find(
    {"loanDate": {"$regex": "09/2025"}},
    {"loanID": 1, "loanDate": 1, "dateTime": 1}
)

for doc in cursor:
    print(doc)

In [ ]:
# Cell 5 - Fetching raw loans from Mongo with Python-side date filter (using dateTime + status)

start = time.time()

# Fetch loans
cursor = Loans.find(
    { 
        "dateTime": {"$gte": date_start, "$lte": date_end},  # filtering by dateTime
    },
    {
        "_id": 0, 
        "loanAmount" : 1,
        "amountPaid" : 1,
        "dueDate" : 1,
        "loanDate" : 1,
        "dueAmount" :1,
        "amountwithInterest" : 1,
        "loanStatus" : 1,
        "interest_rate" : 1,
        "clientID" : 1, 
        "bonusAmount" : 1,
        "penaltyAmounnt": 1,
        "amountDisbursed" : 1,  
        "totalDueAmount" : 1,
        "dateTime": 1
    })

loans_df = pd.DataFrame(list(cursor))

#Check if empty
if loans_df.empty:
    print("No completed loans were made during this period.")
else:
    # Convert string date fields → datetime
    for col in ["loanDate", "dueDate"]:
        loans_df[col] = pd.to_datetime(loans_df[col], dayfirst=True, errors="coerce")

    # Sort refunds by dateTime
    loans_df = loans_df.sort_values("dateTime", ascending=True)
    
# Clean fields
        date_cols = ["loanDate", "dueDate"]
        for col in date_cols:
            loans_df[col] = pd.to_datetime(loans_df[col], dayfirst=True, errors="coerce")

    print("Fetched completed loans:", loans_df.shape)
    print(loans_df.head(3))  # show a small sample

end = time.time()
print(f"Fetching time: {round((end-start)/60, 2)} minutes")        

In [ ]:
# Cell 6 - Clean + filter loans

# Columns with string dates
date_cols = ["loanDate", "dueDate"]
for col in date_cols:
    loans_df[col] = pd.to_datetime(loans_df[col], dayfirst=True, errors="coerce")

# Ensure new fields exist (fill with 0 if missing)
req_fields = ["dueAmount", "penaltyAmount"]
for col in req_fields:
    if col not in loans_df.columns:
        loans_df[col] = 0
    else:
        loans_df[col] = loans_df[col].fillna(0)

# Fill missing numeric values in the standard fields
num_cols = ["loanAmount",
        "amountPaid",
        "amountwithInterest",
        "interest_rate" ,
        "is_complete" , 
        "bonusAmount" ,
        "amountDisbursed", 
        "totalDueAmount"  ]
for col in num_cols:
    if col in loans_df.columns:
        loans_df[col] = loans_df[col].fillna(0)

# Optional: filter by dateTime window again (safety check)
loans_df = loans_df[
    (loans_df["dateTime"] >= date_start) &
    (loans_df["dateTime"] <= date_end)
]

# Sort like the boss does
loans_df = loans_df.sort_values("dateTime", ascending=True)

print("Cleaned loans in date window:", loans_df.shape)
print(loans_df.head(5))

In [ ]:
# Cell 7 - Final cleaning + sample view for Loans

# Convert date fields properly
loans_df["loanDate"] = pd.to_datetime(loans_df["loanDate"], errors="coerce")
loans_df["dueDate"] = pd.to_datetime(loans_df["dueDate"], errors="coerce")

# Ensure numeric fields are clean
numeric_cols = ["loanAmount",
        "amountPaid",
        "amountwithInterest",
        "interest_rate" ,
        "bonusAmount" ,
        "amountDisbursed", 
        "totalDueAmount" ]
for col in numeric_cols:
    loans_df[col] = loans_df[col].fillna(0).apply(lambda x: max(x, 0))

# Clean text fields (IDs, status, refundType, etc.)
text_cols = [
        "loanStatus",
        "clientID"
    ]
for col in text_cols:
    if col in loans_df.columns:
        loans_df[col] = loans_df[col].astype(str).str.strip()

# Reset index for a neat table
loans_df.index = np.arange(loans_df.shape[0])

# Optional: sort by refund date (chronological order)
loans_df = loans_df.sort_values("loanDate", ascending=True)

# Show a random sample of fully cleaned loans
print("Sample of fully cleaned loans:")
display(loans_df.sample(10))


In [9]:
# Printing the payload csv
loans_df.to_csv("loans_may_2025.csv", index=False, sep=",")
print("csv export of may_2025.csv complete")

csv export of may_2025.csv complete
